# Setup

In [2]:

import os
import openai
import requests
 

client = openai.OpenAI(api_key=input())
os.environ["OPENAI_API_KEY"] = input() 
def llm(prompt, stop=["\n"]):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[{'role' : 'user', 'content' : prompt}],
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response.choices[0].message.content

In [3]:
%pip install gym

Note: you may need to restart the kernel to use updated packages.


In [4]:
import scraper, wrappers
env = scraper.WikiSearch(api_key=input())
env = wrappers.HotPotQAWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

# ReAct

In [5]:
import json
import sys

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

# webthink_examples = prompt_dict['webthink_simple6']
# instruction = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
# (1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
# (2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
# (3) Finish[answer], which returns the answer and finishes the task.
# Here are some examples.
# """
# webthink_prompt = instruction + webthink_examples
with open('prompts/team.txt', 'r') as f:
    prompt = '\n'.join(f.readlines())
def webthink(idx=None, prompt=prompt, to_print=True):
    question = env.reset(idx=idx)
    if to_print:
        print(idx, question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        try:
            thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        except Exception as e:
            print(f'too long? bad error {e}')
            return 0, info
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if prompt.count(' ') > 10_000:
            print('TOO LONG TOO LONG')
            return 0, info
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish | NO ANSWER REACHED")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

In [6]:
import random
import time
from tqdm import tqdm
import json

idxs = list(range(7405))
# random.Random(233).shuffle(idxs)

rs = []
infos = []
old_time = time.time()
ID = 10
try:
    for i in tqdm(idxs[ID:ID+1]):
    # for i in tqdm(idxs[758:1000]):
        print(f'INDEX = {i}')
        r, info = webthink(i, to_print=True)
        rs.append(info['em'])
        infos.append(info)
        print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
        print('-----------')
        print()
except:
    print('Died for unknown reason')

with open('758-react_baseline_2.json', 'w') as f:
    json.dump(infos, f)

  0%|          | 0/1 [00:00<?, ?it/s]

INDEX = 10
10 Question: What is the name of the fight song of the university whose main campus is in Lawrence, Kansas and whose branch campuses are in the Kansas City metropolitan area?
Thought 1: I need to find the university with the main campus in Lawrence, Kansas and branch campuses in the Kansas City metropolitan area, then search for the name of their fight song.

Action 1: search | University in Lawrence, Kansas with branch campuses in Kansas City metropolitan area | what is the name of their fight song? | fight song
Observation 1: There is no information in the provided text about the fight song of the University of Kansas. The University of Kansas, founded in 1865, has a total enrollment of 26,708 students across its campuses. The school's sports teams, known as the Kansas Jayhawks, have won thirteen National Championships in various sports.

Thought 2: The text did not provide information about the fight song. I should search for the University of Kansas and specifically look

100%|██████████| 1/1 [00:27<00:00, 27.93s/it]

OURS: im jayhawk
CORRECT: kansas song
Thought 6: The Kansas Jayhawks' fight song is "I'm a Jayhawk." This answers the original question about the name of the fight song of the university with the main campus in Lawrence, Kansas, and branch campuses in the Kansas City metropolitan area. I should finish the task.

Action 6: finish | "I'm a Jayhawk"
Observation 6: Episode finished, reward = 0


{'steps': 6, 'answer': ' "I\'m a Jayhawk"', 'gt_answer': 'Kansas Song', 'question_idx': 10, 'reward': False, 'em': False, 'f1': 0} 

0 1 0.0 27.929442167282104
-----------

